## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### 1. Open the data file and have a look at the general information. 

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/credit_scoring_eng.csv')
pd.set_option('display.max_row',100)
df.info()


# the days_employed are negative
# there exists NaN values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

On first glace I noticed:

-days employed are negative

-there exists NaN values in the days_employed and total_income columns. It would be safe to say that the two columns are correlated because one cannot calculate total_income without days employed. The reason for the missing values could be human error. Maybe the employees are new hires so data does not exist as of yet.

-education needs to be lowercase
-purpose might need to be grouped together?

### 2. Data preprocessing

### Processing missing values

In [3]:
#Finding out sum & percentage of missing values in each feature.
print("======= Amount Missing =======")
missing_values_count = df.isnull().sum()
print(missing_values_count[0:10])
print("======= Percent Missing =======")
null = (df.isna().sum()/len(df)) * 100
print(null[null>0].sort_values())

df.isnull().sum()
df.dropna(inplace=True)


======= Amount Missing =======
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
dtype: int64
======= Percent Missing =======
days_employed    10.099884
total_income     10.099884
dtype: float64


### Conclusion

We drop the missing values because we do not have the data to fill them in. Filling in n/a with the mean value would be wrong. We could have also replaced the n/a values with 0. 2174 lines were dropped in days_employed and total_income.



### Data type replacement

In [4]:
df['days_employed'] = df['days_employed'].astype(int)
df['days_employed'] = df['days_employed'].abs()
df['total_income'] = df['total_income'].astype(int)
df['education'] = df['education'].str.lower()
df['purpose'] = df['purpose'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966,housing transactions
21521,0,343937,67,secondary education,1,married,0,F,retiree,0,24959,purchase of a car
21522,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347,property
21523,3,3112,38,secondary education,1,married,0,M,employee,1,39054,buying my own car


### Conclusion

-days employed is now int instead of float, the absolute value was taken to get positive numbers
-total income was changed to int instead of float because the cents don't really matter
-education was changed to lower to make categorization easier

### Processing duplicates

In [5]:
df2 = df.duplicated().sum()
df.drop_duplicates().reset_index(drop=True)
print(df.dtypes)

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object


### Conclusion

Duplicates if they existed are now dropped from the DataFrame using the drop_duplicates method. The drop_duplicates method was used with the argument "inplace=True" because we want the changes to be made on the original DataFrame. 
df2 = df.duplicated().sum() shows that there are 0 lines in the DataFrame that are duplicated. No lines were deleted.

### Categorizing Data

In [6]:
#correlation between having children and repaying a loan on time


def income_level(row):
    income = int(row['total_income'])
    
    if income < 20000:
        return 'Poverty'
    elif 20000 <= income <= 44000:
        return 'Low Income'
    elif 45000 <= income <= 139000:
        return 'Middle Class'
    elif 140000 <= income <= 149000:
        return 'Upper Middle Class'
    elif 150000 <= income <= 199999:
        return 'High Income'
    else:
        if income >= 200000:
            return 'Highest Tax Bracket'

row_values = ['1500000']
row = pd.Series(data=row_values, index=['total_income'])

income_level(row)

df['income_level'] = df.apply(income_level, axis=1)


def have_kids(new_row):
    kids = int(new_row['children'])
    if kids > 0:
        return '1'
    else:
        return '0'
new_row_values = ['0']
new_row = pd.Series(data=new_row_values, index=['children'])

have_kids(new_row)
df['have_kids'] = df.apply(have_kids, axis=1)

# Creating Pivot Tables

df['kids_ratio'] = df['children']/df['children'].sum()
#df.pivot_table(index=['have_kids', 'kids_ratio'], values='debt')
df['kids_ratio'].count()

#df.pivot_table(index='family_status', values='debt')

#df.pivot_table(index='income_level', values='debt')
#df['income_level'].value_counts()

19351

### Conclusion

a correlation exists between the amount of children one has and the ability to repay a loan. There also exists a correlation between family status and the ability to repay a loan.




### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [7]:
rate = df['debt'].sum() / df.shape[0] #rate for all loans
print("The rate for all loans is:", rate,"%")
print(df['children'].corr(df['debt']))


#noticed a negative value for children
df['children'] = df['children'].replace([-1], 0) #replaces -1 w/ 0
df.groupby(['children'])['debt'].agg(['mean'])['mean']*100

# 20 children seems a little outrageous .. could be an error

The rate for all loans is: 0.0811844349129244 %
0.02135295137788271


children
0      7.472166
1      9.394428
2      9.562399
3      7.482993
4      8.823529
5      0.000000
20    11.940299
Name: mean, dtype: float64

There does not seem to be a relation between having children and defaulting on loans. There does exist an outlier in the data set.

### Conclusion

- Is there a relation between marital status and repaying a loan on time?

In [8]:
df.groupby(['family_status'])['debt'].agg(['mean'])['mean']*100

family_status
civil partnership     9.076305
divorced              7.017544
married               7.592210
unmarried            10.059406
widow / widower       6.473988
Name: mean, dtype: float64

results show that unmarried customers and customers in a civil partnership default more on loans.

### Conclusion

- Is there a relation between income level and repaying a loan on time?

In [9]:
print(df['total_income'].describe())
print("==============================")
print(df.groupby(['income_level'])['debt'].agg(['mean'])['mean']*100)

count     19351.000000
mean      26787.071262
std       16475.452412
min        3306.000000
25%       16488.000000
50%       23202.000000
75%       32549.500000
max      362496.000000
Name: total_income, dtype: float64
income_level
High Income            5.882353
Highest Tax Bracket    9.090909
Low Income             8.239663
Middle Class           7.111597
Poverty                8.250780
Upper Middle Class     0.000000
Name: mean, dtype: float64


The high income bracket has defaulted less on loans than the highest income tax bracket. Low Income and Poverty are about the same.
Having the highest tax bracket be the highest defaulter of loans seems a bit weird.

### Conclusion

- How do different loan purposes affect on-time repayment of the loan?

"If the loan is for school or business, banks might not see a payment on the loan for a while. Versus if the loan is for a house, car, wedding, which are one time things, banks will see a payment after the event has happened"

### Conclusion

### Step 4. General conclusion

Data categorization is used in data to organize data into categories so it is easy to retrieve/sort data.

Using pivot tables and groupby, I was able to determine that there was no relation between having kids and defaulting on a loan. I was also able to determine that there was an outlier in the data which could have been human error. Using groupby I analyzed the relation between marital status and repaying a loan on time and income level and repaying a loan on time.

Different loan purposes affect on-time repayment because as stated in the conclusion above, one-time events such as weddings or car/house purchases will start to see payment after the event; school/business not so much as the customer might need to take out another loan.